# Train and Test MobileNetV1

In [ ]:
! git clone https://github.com/MaxHReinhardt/ArchitecturalStyleClassification

In [ ]:
import sys
import os

py_file_location = "/content/ArchitecturalStyleClassification/src"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
from torchvision.transforms import v2
import torch
import matplotlib.pyplot as plt

from dataset import ArchitecturalStylesDataset
from models import MobileNetV1
from train_model import train_for_n_epochs, train_with_early_stopping
from evaluate_model import evaluate

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip /content/gdrive/MyDrive/data.zip;

In [ ]:
drive.flush_and_unmount()

## Train

In [ ]:
# Train for n epochs

model = MobileNetV1(ch_in=3, n_classes=25, with_cbam=True)

train_transforms = v2.Compose(
    [
        # Convert to image tensor with datatype uint8
        v2.ToImage(),
        v2.ToDtype(torch.uint8, scale=True),
        # Augment data
        v2.RandomResizedCrop(size=(320, 320), antialias=True),
        v2.RandomHorizontalFlip(p=0.5),
        v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=None),
        v2.RandomRotation(degrees=15),
        # v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Apply random affine transformations
        # v2.RandomGrayscale(p=0.1),
        # Convert to float32 and normalize
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and SD
    ]
)

train_set = ArchitecturalStylesDataset(
    csv_file="data/dataset/train_annotation.csv",
    transform=train_transforms,
)

batch_size = 64
learning_rate = 0.003
num_epochs = 10

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model.to(device)

MobileNetV1_cbam_model, train_loss_development = train_for_n_epochs(model, train_set, batch_size, learning_rate, num_epochs, device)
print(f"Losses: {train_loss_development}")

In [ ]:
# Plot Losses
plt.plot(train_loss_development)
plt.xlabel('Number of Training Examples (10 batches a 64 datapoints)')
plt.ylabel('Cross Entropy Error')
plt.title('Plot of Losses')
plt.show()

In [ ]:
# Train with early stopping

model = MobileNetV1(ch_in=3, n_classes=25, with_cbam=True)

train_transforms = v2.Compose(
    [
        # Convert to image tensor with datatype uint8
        v2.ToImage(),
        v2.ToDtype(torch.uint8, scale=True),
        # Augment data
        v2.RandomResizedCrop(size=(320, 320), antialias=True),
        v2.RandomHorizontalFlip(p=0.5),
        v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=None),
        v2.RandomRotation(degrees=15),
        # v2.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Apply random affine transformations
        # v2.RandomGrayscale(p=0.1),
        # Convert to float32 and normalize
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and SD
    ]
)

train_set = ArchitecturalStylesDataset(
    csv_file="data/dataset/train_annotation.csv",
    transform=train_transforms,
)

val_set = ArchitecturalStylesDataset(
    csv_file="data/dataset/validation_annotation.csv",
    transform=train_transforms,
)

batch_size = 64
learning_rate = 0.003
max_num_epochs = 100

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model.to(device)

MobileNetV1_cbam_early_stopping, train_loss_development, val_loss_development = train_with_early_stopping(model, train_set, val_set,
                                                                                        batch_size, learning_rate,
                                                                                        max_num_epochs, device)
print(f"train_loss_development: {train_loss_development}")
print(f"val_loss_development: {val_loss_development}")

In [ ]:
# Plot train and validation loss development
plt.plot(train_loss_development, label='Training Loss', color='blue')
plt.plot(val_loss_development, label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Error')
plt.title('Plot of Losses')
plt.legend()
plt.show()

## Evaluate

In [ ]:
batch_size=64
accuracy, macro_f1, avg_loss = evaluate(MobileNetV1_cbam_model, train_set, batch_size, device)  # Use train set also for evaluation for testing
print(f"Accuracy on Train Set: {accuracy}")
print(f"Macro F1 on  Train Set: {macro_f1}")

In [ ]:
validation_set = ArchitecturalStylesDataset(
    csv_file="data/dataset/validation_annotation.csv",
    transform=train_transforms,
)

accuracy, macro_f1, avg_loss = evaluate(MobileNetV1_cbam_model, validation_set, batch_size, device)  # Use train set also for evaluation for testing
print(f"Accuracy on validation set: {accuracy}")
print(f"Macro F1 on  validation set: {macro_f1}")
print(f"Average loss on validation set: {avg_loss}")